<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/AI_generated_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.7

In [2]:
import pandas as pd
import numpy as np
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [17]:
IC_DF = pd.read_csv(r'https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Text_Classification_Results.csv').drop(['Unnamed: 0'], axis='columns')

def Preprocess(df):
  cols = list(df.columns) 
  to_drop = []
  ohe = []
  for col in cols:
    if type(df[col][0]) == str:
      if len(np.unique(df[col])) == 1:
        to_drop.append(col)
      else:
        ohe.append(col)
    if type(df[col][0]) == np.bool_:
      boolys = []
      for i in range(len(df[col])):
        if df[col][i] == True:
          boolys.append(1)
        if df[col][i] == False:
          boolys.append(0)
      df[col] = boolys
  df2 = df.drop(to_drop, axis='columns')
  try:
    df3 = pd.get_dummies(df[ohe])
  except:
    return df2
  return pd.concat([df2.drop(ohe, axis='columns'), df3], axis=1)

In [18]:
IC_DF.shape

(8, 18)

In [19]:
Preprocess(IC_DF)

,output_dim,embedding,nodes,regularizer,stacking,dropout,bidirectional,epochs,batch_size,accuracy,loss,val_accuracy,val_loss,len_dataset,n_features
0,3,12300,64,NaN,0,0,0,6,32,0.832541,0.340803,0.601230,0.976570,8936,17
1,3,12300,64,NaN,1,0,0,7,32,0.838976,0.342386,0.563758,1.000242,8936,17
2,3,12300,64,L1,0,0,0,6,32,0.837437,0.328798,0.585011,0.999480,8936,17
3,3,12300,64,L1,1,0,0,7,32,0.838137,0.310724,0.579978,1.231099,8936,17
4,2,45589,64,NaN,0,0,0,9,32,1.000000,0.018424,0.569444,1.241262,360,1724
5,2,45589,64,NaN,0,1,0,9,32,0.625000,0.908849,0.000000,2.004611,360,1724
6,2,45589,64,NaN,1,0,0,6,32,0.625000,1.255753,0.000000,2.963903,360,1724
7,2,45589,64,NaN,1,1,0,6,32,0.625000,1.148167,0.000000,2.705530,360,1724


In [15]:
IC_DF

,nodes,activation,regularizer,stacking,dropout,nlayers,closer,loss_func,y_col,epochs,batch_size,optimizer,loss,val_loss,len_dataset,n_features
0,32,relu,NaN,0,0,4,0,MSE,string,18,32,rmsprop,59.181824,44.038368,506,13
1,32,relu,NaN,0,0,4,0,MAE,string,11,32,rmsprop,75.885368,94.594131,506,13
2,64,relu,NaN,0,0,4,0,MSE,string,7,32,rmsprop,63.254677,30.891106,506,13
3,64,relu,NaN,0,0,4,0,MAE,string,33,32,rmsprop,69.140221,30.245502,506,13
4,128,relu,NaN,0,0,4,0,MSE,string,18,32,rmsprop,70.252892,41.010700,506,13
5,128,relu,NaN,0,0,4,0,MAE,string,11,32,rmsprop,58.902702,99.228020,506,13
